In [95]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import random
from matplotlib import image
from sklearn.model_selection import train_test_split
import pickle
from datetime import datetime
from sklearn.utils.class_weight import compute_class_weight
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from prettytable import PrettyTable 
from contextlib import redirect_stdout
from sklearn.metrics import auc
import time

In [96]:
# RUN MODELS 
%run models.ipynb

In [97]:
# DEFINITIONS
TRAIN_DIR_PATH = '../Alzheimer_s Dataset/train'
TEST_DIR_PATH = '../Alzheimer_s Dataset/test'
RESULT_PATH = 'results'
SEED = 42
DATASET_PERCENTAGE = 1
VALIDATION_PERCENTAGE = 0
N_CLASSES = 4
VERBOSE = {
    'ANALYSIS': False,
    'LOADING': False,
    'SPLITTING': False,
    'EXECUTION': False,
    'EVALUATION': False
}

In [98]:
def data_analysis_histogram(classes):
    class_dist = []
    for c in classes:
        class_path = os.path.join(TRAIN_DIR_PATH,c)
        class_dist.append(len(os.listdir(class_path)))
    
    if VERBOSE['ANALYSIS']:
        plt.figure(figsize=(16, 8))
        plt.title("Class distribution")
        plt.barh(classes, class_dist)
        for index, value in enumerate(class_dist):
            plt.text(value, index,str(value))
        plt.show()
    return class_dist

def data_analysis_image_size(classes):
    random.seed(SEED)
    random_class_path = os.path.join(TRAIN_DIR_PATH,random.choice(classes))
    random_img_name = random.choice(os.listdir(random_class_path))
    random_img_path = os.path.join(random_class_path,random_img_name)
    img = image.imread(random_img_path)
    if VERBOSE['ANALYSIS']:
        plt.figure(figsize=(16, 8))
        plt.title("%s - Height: %d px x Length: %d px" % (random_img_path,img.shape[0],img.shape[1]))
        plt.imshow(img)
    
    return (img.shape[0],img.shape[1],1)

def analyse_dataset():
    classes = os.listdir(TRAIN_DIR_PATH)
    class_dist = data_analysis_histogram(classes)
    input_shape = data_analysis_image_size(classes)
    return classes, input_shape, class_dist

In [99]:
def load_dataset(dir_path):
    classes = os.listdir(TRAIN_DIR_PATH)
    img_array = []
    class_array = []
    for c in classes:
        class_path = os.path.join(dir_path,c)
        imgs_name = os.listdir(class_path)

        if DATASET_PERCENTAGE < 1:
            imgs_name = random.sample(imgs_name, k = int(len(imgs_name)*DATASET_PERCENTAGE))

        for i in imgs_name:
            img_array.append(image.imread(os.path.join(class_path,i)))
            class_array.append(c)
    if VERBOSE['LOADING']:
        print("Loaded %d images" % len(img_array))
        
    return np.array(img_array), np.array(class_array)

In [100]:
def split_dataset(x, y):
    if VALIDATION_PERCENTAGE <= 0:
        return x, None, y, None

    x_train, x_val, y_train, y_val = train_test_split(x,  y, test_size=VALIDATION_PERCENTAGE, random_state=SEED)
    if VERBOSE['SPLITTING']:
        print("Train size: %d\nValidation size: %d" % (len(x_train), len(x_val)))
    return x_train, x_val, y_train, y_val

In [101]:
def prepare_dataset_channel_position(x, input_shape):
    img_lin,img_col,n_channels = input_shape
    if keras.backend.image_data_format() == 'channels_first':
        x = x.reshape(x.shape[0], n_channels, img_lin, img_col)
        input_shape = (n_channels, img_lin, img_col)
    else:
        x = x.reshape(x.shape[0], img_lin, img_col, n_channels)
        input_shape = (img_lin, img_col, n_channels)
    return x, input_shape

def prepare_dataset_input(x, input_shape):
    x_scaled = x.astype('float32') / 255.0
    return prepare_dataset_channel_position(x_scaled, input_shape)

def prepare_dataset_output(y, classes):
    class_map = {x: i for i,x in enumerate(classes)}
    y_code = [class_map[word] for word in y]
    y_categorical = keras.utils.to_categorical(y_code, len(classes))
    inv_class_map = {v: k for k, v in class_map.items()}
    return y_categorical, inv_class_map

def prepare_dataset(x , y , classes, input_shape):
    if x is None:
        return None, None, None, None
    x_scaled, input_shape = prepare_dataset_input(x, input_shape)
    y_categorical, inv_class_map = prepare_dataset_output(y, classes)
    return x_scaled , y_categorical, inv_class_map, input_shape

In [102]:
def save_model(model, history, elapsed_minutes):
    result_directory = os.path.join(RESULT_PATH)

    if not os.path.exists(result_directory):
        os.makedirs(result_directory)
    
    name = model.name

    result_directory = os.path.join(result_directory,name)

    if not os.path.exists(result_directory):
        os.makedirs(result_directory)
    else:
        raise ValueError("File already exists.")
    
    model_path = os.path.join(result_directory,'model')
    model.save(model_path)

    execution_path = os.path.join(result_directory,'execution')

    execution = {
            'epochs': history.params['epochs'],
            'history': history.history,
            'elapsed_minutes': elapsed_minutes
    }

    with open(execution_path, 'wb') as f:
        pickle.dump(execution, f)
    
    pretty_model_path = os.path.join(result_directory, 'model_summary.txt')

    with open(pretty_model_path,'w') as f:
        f.write('Elapsed time: '+str(elapsed_minutes)+' min\n')
        with redirect_stdout(f):
            model.summary()

    return name

In [103]:
def load_model(model_name):
    result_directory = os.path.join(RESULT_PATH, model_name)
    if not os.path.exists(result_directory):
        raise ValueError("Folder not found.")
    
    model_path = os.path.join(result_directory,'model')
    model = keras.models.load_model(model_path)

    execution_path = os.path.join(result_directory,'execution')
    execution = pickle.load(open(execution_path, "rb"))
    
    return model, execution

In [104]:
def save_evaluation(model_name, evalution_name, all_scores, table):
    result_directory = os.path.join(RESULT_PATH,model_name)
    if not os.path.exists(result_directory):
        raise ValueError("Folder not found.")
    
    score_path = os.path.join(result_directory, evalution_name + '_score.json')
    with open(score_path, 'w') as f:
        json.dump(all_scores, f, ensure_ascii=False, indent=4)

    pretty_score_path = os.path.join(result_directory, evalution_name + '_score_summary.txt')

    with open(pretty_score_path,'w') as f:
        f.write(model_name+'\n')
        f.write(table.get_string())

In [105]:
def plot_evaluation_roc(evaluation_name, model, x, y, inv_class_map):
    y_pred = model.predict(x)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(inv_class_map)):
        fpr[i], tpr[i], _ = roc_curve(y[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    plt.figure(figsize=(16, 16))
    for i in range(len(inv_class_map)):
        plt.plot(
            fpr[i],
            tpr[i],
            label="ROC curve of {0} (AUC = {1:0.2f})".format(inv_class_map[i], roc_auc[i]))
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")

    result_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(result_directory,evaluation_name+'_roc_curve.png')
    plt.savefig(image_path)

    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
        
    return fpr, tpr, roc_auc
    
def evaluate_model_by_class(model, x, y):
    def separate_by_class(x, y):
        n_classes = y.shape[1]
        x_classified = [[] for _ in range(n_classes)]
        y_classified = [[] for _ in range(n_classes)]
        
        for i,img in enumerate(y):
            index = np.where(img==1)[0][0]
            x_classified[index].append(x[i])
            y_classified[index].append(y[i])

        for i in range(n_classes):
            x_classified[i] = np.array(x_classified[i])
            y_classified[i] = np.array(y_classified[i])
            
        return np.array(x_classified,dtype=object), np.array(y_classified,dtype=object)

    x_by_class, y_by_class = separate_by_class(x,y)
    
    score_by_class = []
    for x,y in zip(x_by_class,y_by_class):
        score = model.evaluate(x, y, verbose = 1 if VERBOSE['EVALUATION'] else 0)
        score_by_class.append(score)

    return score_by_class

def evaluate_model_confusion_matrix(evaluation_name, model, x, y, inv_class_map):
    def undoOneHotEncoding(y, inv_class_map):
        return [inv_class_map[i] for i in np.argmax(y, axis = 1)]
    
    y_pred = model.predict(x)
    y_pred_int = undoOneHotEncoding(y_pred, inv_class_map)
    y_int = undoOneHotEncoding(y, inv_class_map)
    cm = confusion_matrix(y_int, y_pred_int)

    cm_df = pd.DataFrame(cm,
                        index = [inv_class_map[i] for i in range(len(inv_class_map))], 
                        columns = [inv_class_map[i] for i in range(len(inv_class_map))])
    plt.figure(figsize=(16,16))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    
    results_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(results_directory,evaluation_name+'_confusion_matrix.png')
    plt.savefig(image_path)
    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
    return cm

def evaluate_model(evaluation_name, model, x, y, inv_class_map):
    if x is None:
        return None

    score = model.evaluate(x, y, verbose = 1 if VERBOSE['EVALUATION'] else 0)
    score_by_class = evaluate_model_by_class(model, x, y)
    cm = evaluate_model_confusion_matrix(evaluation_name, model,x, y, inv_class_map)
    _, _, roc_auc = plot_evaluation_roc(evaluation_name, model, x, y, inv_class_map)
    table = PrettyTable()

    table.add_column("Metrics", model.metrics_names)
    table.add_column("Global", np.round(score,4))

    for i, s_class in enumerate(score_by_class):
        table.add_column(inv_class_map[i], np.round(s_class,4))

    if VERBOSE['EVALUATION']:
        print()
        print(evaluation_name)
        print(table)

    all_scores = {
        'model_name': model.name,
        'name': evaluation_name,
        'loss': score,
        'loss_by_class': score_by_class,
        'confusion': cm.tolist(),
        'auc': list(roc_auc.values())
    }

    save_evaluation(model.name, evaluation_name, all_scores, table)

    return all_scores

In [106]:
def get_class_weight(classes, y):
    class_weight = compute_class_weight(class_weight ='balanced', classes = classes, y = y)
    return dict(zip(range(len(classes)),class_weight))

In [107]:
def get_metrics():
    return [
        keras.metrics.Accuracy(name="Accuracy"),
        keras.metrics.Precision(name='Precision'),
        keras.metrics.Recall(name='Recall'),
        keras.metrics.AUC(name='AUC'),
        keras.metrics.AUC(name='PRC', curve='PR'), # precision-recall curve
    ]

In [108]:
def plot_execution_history(history, model):
    result_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(result_directory,'execution.png')

    if VALIDATION_PERCENTAGE > 0:
        nplots = len(history.values())/2
    else:
        nplots = len(history.values())
    nrows = int(nplots/3)
    ncols = 3
    fig = plt.figure(figsize=(ncols*8, nrows*5))
    gs = fig.add_gridspec(nrows, ncols)
    axs = gs.subplots(sharex=False, sharey=False)
    for i,h in enumerate(history.values()):
        i_hat = int(i%nplots)
        r = int(i_hat/ncols)
        c = i_hat%ncols
        if VALIDATION_PERCENTAGE > 0:
            axs[r,c].plot(h, label = 'Training' if int(i/nplots)==0 else 'Validation')
        else:
            axs[r,c].plot(h, label = 'Training')
        
    for i,n in enumerate(model.metrics_names):
        r = int(i/ncols)
        c = i%ncols
        axs[r,c].set_xlabel('Epoch')
        axs[r,c].set_ylabel(n)
        axs[r,c].legend()
      
    plt.savefig(image_path)
    if VERBOSE['EXECUTION']:
        plt.show()
    else:
        plt.close()
        
def run_model(model, x_train, y_train, x_val, y_val, class_weight):
    if VERBOSE['EXECUTION']:
        model.summary()

    model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=get_metrics(), weighted_metrics=get_metrics())

    if x_val is None:
        xy_val = None
    else:
        xy_val = (x_val, y_val)

    start_time = time.time()
    history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data= xy_val,
                    class_weight = class_weight,
                    verbose=1 if VERBOSE['EXECUTION'] else 0)

    elapsed_minute = round((time.time() - start_time)/60)

    save_model(model, history, elapsed_minute)
    plot_execution_history(history.history, model)
      
    return model, history

In [109]:
def train_and_evaluate(models_func):
    classes, input_shape, _ = analyse_dataset()
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)
    x_train, x_val, y_train, y_val = split_dataset(x_trainval, y_trainval)
    x_train_prepared , y_train_prepared, inv_class_map, input_shape = prepare_dataset(x_train , y_train , classes, input_shape)
    x_val_prepared , y_val_prepared, _, _ = prepare_dataset(x_val , y_val , classes, input_shape)
    x_test_prepared , y_test_prepared, _, _ = prepare_dataset(x_test , y_test , classes, input_shape)
    for model_func in models_func:
        model, history = run_model(model_func(input_shape, classes), x_train_prepared, y_train_prepared, x_val_prepared, y_val_prepared, get_class_weight(classes, y_train))
        history = history.history
        evaluate_model('training',model, x_train_prepared, y_train_prepared, inv_class_map)
        evaluate_model('validation', model, x_val_prepared, y_val_prepared, inv_class_map)
        evaluate_model('test', model, x_test_prepared, y_test_prepared, inv_class_map)


def reevaluate():
    classes, input_shape, _ = analyse_dataset()
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)
    x_train, x_val, y_train, y_val = split_dataset(x_trainval, y_trainval)
    x_train_prepared , y_train_prepared, inv_class_map, input_shape = prepare_dataset(x_train , y_train , classes, input_shape)
    x_val_prepared , y_val_prepared, _, _ = prepare_dataset(x_val , y_val , classes, input_shape)
    x_test_prepared , y_test_prepared, _, _ = prepare_dataset(x_test , y_test , classes, input_shape)
    for f in os.listdir(RESULT_PATH):   
        model, _ = load_model(f)
        evaluate_model('training',model, x_train_prepared, y_train_prepared, inv_class_map)
        evaluate_model('validation', model, x_val_prepared, y_val_prepared, inv_class_map)
        evaluate_model('test', model, x_test_prepared, y_test_prepared, inv_class_map)

In [110]:
stacked_models_0 = [  
                    classweight,
                    classweight_3_conv2d_layers,
                    classweight_4_conv2d_layers,
                    classweight_5_conv2d_layers,
                    classweight_6_conv2d_layers,
                    classweight_7_conv2d_layers,
                ]
stacked_models_1 = [  
                    classweight_last_conv2d,
                    classweight_last_conv2d_3_conv2d_layers,
                    classweight_last_conv2d_4_conv2d_layers,
                    classweight_last_conv2d_5_conv2d_layers,
                    classweight_last_conv2d_6_conv2d_layers,
                    classweight_last_conv2d_7_conv2d_layers,
                ]

stacked_models_2 = [
                    classweight_1_conv2d_layers,
                    classweight_last_conv2d_1_conv2d_layer,
                    classweight_no_pooling_1_conv2d_layer,
                    classweight_no_pooling_2_conv2d_layers,
                    classweight_no_pooling_3_conv2d_layers
                ]
#stacked_train_and_evaluate(stacked_models_2)
reevaluate()